In [ ]:
from collections import OrderedDict as odict
from pathlib import Path
import geopandas as gpd

import colorcet
import param
import panel
import datashader as ds
import intake
import dask.dataframe

import spatialpandas
import cartopy

import holoviews
import geoviews
from holoviews.element import tiles
from holoviews.operation.datashader import rasterize, shade, spread
from holoviews.element.tiles import EsriStreet

import seaborn
import datashader.transfer_functions as tf

holoviews.extension('bokeh', logo=False)

In [ ]:
%%time
gdf = gpd.read_file(Path('data').joinpath('National_Forest_Inventory_Woodland_GB_2021_10ksample.geojson'))

In [ ]:
%%time
target = 'IFT_IOA'
gdf['categories'] = gdf[target].astype('category').cat.codes
gdf = gdf.to_crs("epsg:3395")

In [ ]:
fields = gdf[target].unique().tolist()

cmaps = odict([(n, colorcet.palette[n]) for n in ['fire', 'bgy', 'bgyw', 'bmy', 'gray', 'kbc']])

maps = ['OSM', 'CartoDark', 'OpenTopoMap']

bases = odict([(name, getattr(tiles, name)().relabel(name)) for name in maps])

gopts = holoviews.opts.Tiles(responsive=True, xaxis=None, yaxis=None, show_grid=True)

class Explorer(param.Parameterized):

    field = param.Selector(fields)
    
    cmap = param.Selector(cmaps)
    
    spreading = param.Integer(0, bounds=(0, 5))
    
    basemap = param.Selector(bases)
    
    data_opacity = param.Magnitude(1.00)
    
    map_opacity = param.Magnitude(0.75)
    
    show_labels = param.Boolean(True)

    @param.depends('map_opacity', 'basemap')
    def tiles(self):
        return self.basemap.opts(gopts).opts(alpha=self.map_opacity)

    @param.depends('show_labels')
    def labels(self):
        return tiles.StamenLabels().options(level='annotation', alpha=1 if self.show_labels else 0)

    def viewable(self,**kwargs):
        polygons = holoviews.Polygons(gdf, vdims=['categories', target, 'Area_ha']).opts(width=600, height=600, colorbar=True)
     
        rasterized = rasterize(polygons)
        
        shaded = shade(rasterized, cmap=self.param.cmap)
        
        spreaded = spread(shaded, px=self.param.spreading, how="add")
        
        dataplot = spreaded.apply.opts(alpha=self.param.data_opacity, show_legend=False)
        
        return holoviews.DynamicMap(self.tiles) * dataplot * holoviews.DynamicMap(self.labels)

In [ ]:
%%time
explorer = Explorer(name="")

dash = panel.Row(panel.Column(panel.Param(explorer.param, expand_button=False)), explorer.viewable())
dash.servable("Datashader Dashboard")

# logo = "https://raw.githubusercontent.com/pyviz/datashader/main/doc/_static/logo_horizontal_s.png"
# dash = panel.Row(panel.Column(logo, panel.Param(explorer.param, expand_button=False)), explorer.viewable())